In [1]:
import os 
from dotenv import load_dotenv 
load_dotenv()

True

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma

c:\Users\91800\Desktop\GENAI UDEMY\genai-uv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from huggingface_hub import InferenceClient

class MiniLMEmbedding:
    def __init__(self, api_key, model="sentence-transformers/all-MiniLM-L6-v2"):
        self.client = InferenceClient(
            provider="hf-inference",
            api_key=api_key
        )
        self.model = model

    def embed_documents(self, texts):
        vectors = self.client.feature_extraction(
            texts,
            model=self.model,
        )
        return [list(map(float, v)) for v in vectors]

    def embed_query(self, text):
        vectors = self.client.feature_extraction(
            text,
            model=self.model,
        )
        return list(map(float, vectors[0]))


In [4]:
from langchain_core.documents import Document
documents = [
    Document(
        page_content="John F. Kennedy served as the 35th president of the United States from 1961 until his assassination in 1963.",
        metadata={"source": "us-presidents-doc"},
    ),
    Document(
        page_content="Robert F. Kennedy was a key political figure and served as the U.S. Attorney General; he was also assassinated in 1968.",
        metadata={"source": "us-politics-doc"},
    ),
    Document(
        page_content="The Kennedy family is known for their significant influence in American politics and their extensive philanthropic efforts.",
        metadata={"source": "kennedy-family-doc"},
    ),
    Document(
        page_content="Edward M. Kennedy, often known as Ted Kennedy, was a U.S. Senator who played a major role in American legislation over several decades.",
        metadata={"source": "us-senators-doc"},
    ),
    Document(
        page_content="Jacqueline Kennedy Onassis, wife of John F. Kennedy, was an iconic First Lady known for her style, poise, and dedication to cultural and historical preservation.",
        metadata={"source": "first-lady-doc"},
    ),
]

In [5]:

embedding_model2=GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")
embedding2 = MiniLMEmbedding(api_key=os.environ["HF_THIRD_TOKEN"])


In [70]:
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings

embedding=GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")

In [6]:
from huggingface_hub import InferenceClient
from langchain.embeddings.base import Embeddings
import numpy as np

class MiniLMEmbedding(Embeddings):
    def __init__(self, api_key, model="sentence-transformers/all-MiniLM-L6-v2"):
        self.client = InferenceClient(
            provider="hf-inference",
            api_key=api_key
        )
        self.model = model

    def embed_documents(self, texts):
        # returns numpy array (N, D)
        raw = self.client.feature_extraction(texts, model=self.model)
        raw = np.array(raw, dtype=float)
        return raw.tolist()

    def embed_query(self, text):
        # returns numpy array (D,)
        raw = self.client.feature_extraction(text, model=self.model)
        raw = np.array(raw, dtype=float)
        return raw.tolist()


In [ ]:
embedding = MiniLMEmbedding(api_key=os.environ["HF_THIRD_TOKEN"])

vector_store = FAISS.from_documents(
    documents=documents,
    embedding=embedding,
)


In [12]:
vector_store.similarity_search("Who served as the 35th president?")


[Document(metadata={'source': 'us-presidents-doc'}, page_content='John F. Kennedy served as the 35th president of the United States from 1961 until his assassination in 1963.'),
 Document(metadata={'source': 'first-lady-doc'}, page_content='Jacqueline Kennedy Onassis, wife of John F. Kennedy, was an iconic First Lady known for her style, poise, and dedication to cultural and historical preservation.'),
 Document(metadata={'source': 'us-politics-doc'}, page_content='Robert F. Kennedy was a key political figure and served as the U.S. Attorney General; he was also assassinated in 1968.'),
 Document(metadata={'source': 'us-senators-doc'}, page_content='Edward M. Kennedy, often known as Ted Kennedy, was a U.S. Senator who played a major role in American legislation over several decades.')]

In [13]:
retriever = vector_store.as_retriever()

docs = retriever.get_relevant_documents("Who is John F. Kennedy?")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

response = llm.invoke(
    f"Answer using ONLY this context:\n\n{docs}\n\nQuestion: Who was JFK?"
)

print(response)


C:\Users\91800\AppData\Local\Temp\ipykernel_19216\4122207549.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  docs = retriever.get_relevant_documents("Who is John F. Kennedy?")


content='JFK was John F. Kennedy, who served as the 35th president of the United States from 1961 until his assassination in 1963.' response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-23b2004a-ef8c-4247-8d8b-745c7df87479-0'


In [14]:
vector_store.similarity_search("Who was JFK?")


[Document(metadata={'source': 'us-politics-doc'}, page_content='Robert F. Kennedy was a key political figure and served as the U.S. Attorney General; he was also assassinated in 1968.'),
 Document(metadata={'source': 'us-presidents-doc'}, page_content='John F. Kennedy served as the 35th president of the United States from 1961 until his assassination in 1963.'),
 Document(metadata={'source': 'us-senators-doc'}, page_content='Edward M. Kennedy, often known as Ted Kennedy, was a U.S. Senator who played a major role in American legislation over several decades.'),
 Document(metadata={'source': 'kennedy-family-doc'}, page_content='The Kennedy family is known for their significant influence in American politics and their extensive philanthropic efforts.')]

# Using this as Runnables

In [19]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

model=ChatGoogleGenerativeAI(model="gemini-2.5-flash")


In [20]:

message="""
Answer this question using the provided context only.

{question}

Context:
{context}


"""

prompt=ChatPromptTemplate.from_messages([("human",message)])

chain = {"context":retriever,"question":RunnablePassthrough()} | prompt | model 

In [21]:
response = chain.invoke("tell me about Jackie")

print(response.content)

Jacqueline Kennedy Onassis, wife of John F. Kennedy, was an iconic First Lady known for her style, poise, and dedication to cultural and historical preservation.
